# Traffic Signs Detection with YOLO v3, OpenCV and Keras

# 📥 Importing needed libraries

In [1]:
import numpy as np 
import pandas as pd 
import cv2
import time
from timeit import default_timer as timer
import matplotlib.pyplot as plt
import pickle
from keras.models import load_model
import os


Using TensorFlow backend.


# 📂 Loading *labels*

In [2]:
# Reading csv file with labels' names
# Loading two columns [0, 1] into Pandas dataFrame
labels = pd.read_csv('C:/Users/palash/Downloads/datasets_327959_657000_label_names.csv')

# Check point
# Showing first 5 rows from the dataFrame
print(labels.head())
print()

print(labels.iloc[0][1])  # Speed limit (20km/h)
# ***['SignName'][1] - returns element on the column with name 'SignName' and 1 row
print(labels['SignName'][1]) # Speed limit (30km/h)


   ClassId              SignName
0        0  Speed limit (20km/h)
1        1  Speed limit (30km/h)
2        2  Speed limit (50km/h)
3        3  Speed limit (60km/h)
4        4  Speed limit (70km/h)

Speed limit (20km/h)
Speed limit (30km/h)


# 📍 Loading trained Keras CNN model for Classification

In [3]:
# Loading trained CNN model to use it later when classifying from 4 groups into one of 43 classes
model = load_model('C:/Users/palash/Downloads/model (1).h5')

#with open('../input/traffic-signs-preprocessed/mean_image_rgb.pickle', 'rb') as f:
    #mean = pickle.load(f, encoding='latin1')
    
#print(mean['mean_image_rgb'].shape)


# 💠 Loading YOLO v3 network by OpenCV dnn library

## Loading *trained weights* and *cfg file* into the Network

In [4]:
path_to_weights = 'C:/yolov3_ts.weights'
path_to_cfg = 'C:/Users/palash/Downloads/yolov3_ts_test (2).cfg'

# Loading trained YOLO v3 weights and cfg configuration file by 'dnn' library from OpenCV
network = cv2.dnn.readNetFromDarknet(path_to_cfg, path_to_weights)

network.setPreferableBackend(cv2.dnn.DNN_BACKEND_OPENCV)
network.setPreferableTarget(cv2.dnn.DNN_TARGET_OPENCL_FP16)


## Getting *output layers* where detections are made

In [5]:
layers_all = network.getLayerNames()
# Getting only detection YOLO v3 layers that are 82, 94 and 106
layers_names_output = [layers_all[i[0] - 1] for i in network.getUnconnectedOutLayers()]
# Check point
print()
print(layers_names_output)  # ['yolo_82', 'yolo_94', 'yolo_106']



['yolo_82', 'yolo_94', 'yolo_106']


## Setting *probability*, *threshold* and *colour* for bounding boxes

In [6]:
# Minimum probability to eliminate weak detections
probability_minimum = 0.2

# Setting threshold to filtering weak bounding boxes by non-maximum suppression
threshold = 0.2

# Generating colours for bounding boxes
colours = np.random.randint(0, 255, size=(len(labels), 3), dtype='uint8')

print(type(colours))  
print(colours.shape)  
print(colours[0])  

<class 'numpy.ndarray'>
(43, 3)
[168 249  73]


#  Reading input video

In [7]:
# Reading video from a file by VideoCapture object
video = cv2.VideoCapture('../input/traffic-signs-dataset-in-yolo-format/traffic-sign-to-test.mp4')

# Writer that will be used to write processed frames
writer = None

# Variables for spatial dimensions of the frames
h, w = None, None


# Processing frames in the loop

In [18]:
%matplotlib inline

# Setting default size of plots
plt.rcParams['figure.figsize'] = (3, 3)

# Variable for counting total amount of frames
f = 0

# Variable for counting total processing time
t = 0

# Catching frames in the loop
while True:
    # Capturing frames one-by-one
    ret, frame = video.read()

    # If the frame was not retrieved
    if not ret:
        break
       
    # Getting spatial dimensions of the frame for the first time
    if w is None or h is None:
        # Slicing two elements from tuple
        h, w = frame.shape[:2]

    # Blob from current frame
    blob = cv2.dnn.blobFromImage(frame, 1 / 255.0, (416, 416), swapRB=True, crop=False)

    # Forward pass with blob through output layers
    network.setInput(blob)
    start = time.time()
    output_from_network = network.forward(layers_names_output)
    end = time.time()

    # Increasing counters
    f += 1
    t += end - start

    # Spent time for current frame
    print('Frame number {0} took {1:.5f} seconds'.format(f, end - start))

    # Lists for detected bounding boxes, confidences and class's number
    bounding_boxes = []
    confidences = []
    class_numbers = []

    # Going through all output layers after feed forward pass
    for result in output_from_network:
        # Going through all detections from current output layer
        for detected_objects in result:
            # Getting 80 classes' probabilities for current detected object
            scores = detected_objects[5:]
            # Getting index of the class with the maximum value of probability
            class_current = np.argmax(scores)
            # Getting value of probability for defined class
            confidence_current = scores[class_current]

            # Eliminating weak predictions by minimum probability
            if confidence_current > probability_minimum:
                # Scaling bounding box coordinates to the initial frame size
                box_current = detected_objects[0:4] * np.array([w, h, w, h])

                # Getting top left corner coordinates
                x_center, y_center, box_width, box_height = box_current
                x_min = int(x_center - (box_width / 2))
                y_min = int(y_center - (box_height / 2))

                # Adding results into prepared lists
                bounding_boxes.append([x_min, y_min, int(box_width), int(box_height)])
                confidences.append(float(confidence_current))
                class_numbers.append(class_current)
                

    # Implementing non-maximum suppression of given bounding boxes
    results = cv2.dnn.NMSBoxes(bounding_boxes, confidences, probability_minimum, threshold)

    # Checking if there is any detected object been left
    if len(results) > 0:
        # Going through indexes of results
        for i in results.flatten():
            # Bounding box coordinates, its width and height
            x_min, y_min = bounding_boxes[i][0], bounding_boxes[i][1]
            box_width, box_height = bounding_boxes[i][2], bounding_boxes[i][3]
            
            
            # Cut fragment with Traffic Sign
            c_ts = frame[y_min:y_min+int(box_height), x_min:x_min+int(box_width), :]
            # print(c_ts.shape)
            
            if c_ts.shape[:1] == (0,) or c_ts.shape[1:2] == (0,):
                pass
            else:
                # Getting preprocessed blob with Traffic Sign of needed shape
                blob_ts = cv2.dnn.blobFromImage(c_ts, 1 / 255.0, size=(32, 32), swapRB=True, crop=False)
                blob_ts[0] = blob_ts[0, :, :, :] - mean['mean_image_rgb']
                blob_ts = blob_ts.transpose(0, 2, 3, 1)
                # plt.imshow(blob_ts[0, :, :, :])
                # plt.show()

                # Feeding to the Keras CNN model to get predicted label among 43 classes
                scores = model.predict(blob_ts)

                # Scores is given for image with 43 numbers of predictions for each class
                # Getting only one class with maximum value
                prediction = np.argmax(scores)
                # print(labels['SignName'][prediction])


                # Colour for current bounding box
                colour_box_current = colours[class_numbers[i]].tolist()

                # Drawing bounding box on the original current frame
                cv2.rectangle(frame, (x_min, y_min),
                              (x_min + box_width, y_min + box_height),
                              colour_box_current, 2)

                # Preparing text with label and confidence for current bounding box
                text_box_current = '{}: {:.4f}'.format(labels['SignName'][prediction],
                                                       confidences[i])

                # Putting text with label and confidence on the original image
                cv2.putText(frame, text_box_current, (x_min, y_min - 5),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.5, colour_box_current, 2)


    # Initializing writer only once
    if writer is None:
        fourcc = cv2.VideoWriter_fourcc(*'mp4v')

        # Writing current processed frame into the video file
        writer = cv2.VideoWriter('result.mp4', fourcc, 30,
                                 (frame.shape[1], frame.shape[0]), True)

    # Write processed current frame to the file
    writer.write(frame)


# Releasing video reader and writer
video.release()
writer.release()


Frame number 1 took 1.39053 seconds
Frame number 2 took 1.11543 seconds
Frame number 3 took 1.09141 seconds
Frame number 4 took 1.09869 seconds
Frame number 5 took 1.10503 seconds
Frame number 6 took 1.10214 seconds
Frame number 7 took 1.12371 seconds
Frame number 8 took 1.10542 seconds
Frame number 9 took 1.11623 seconds
Frame number 10 took 1.11995 seconds
Frame number 11 took 1.10640 seconds
Frame number 12 took 1.11359 seconds
Frame number 13 took 1.13821 seconds
Frame number 14 took 1.09765 seconds
Frame number 15 took 1.09825 seconds
Frame number 16 took 1.14865 seconds
Frame number 17 took 1.24434 seconds
Frame number 18 took 1.13595 seconds
Frame number 19 took 1.09914 seconds
Frame number 20 took 1.28085 seconds
Frame number 21 took 1.24353 seconds
Frame number 22 took 1.23312 seconds
Frame number 23 took 1.13254 seconds
Frame number 24 took 1.10308 seconds
Frame number 25 took 1.19745 seconds
Frame number 26 took 1.18168 seconds
Frame number 27 took 1.13057 seconds
Frame numb

## 🏁 FPS results

In [19]:
print('Total number of frames', f)
print('Total amount of time {:.5f} seconds'.format(t))
print('FPS:', round((f / t), 1))


Total number of frames 56
Total amount of time 63.45080 seconds
FPS: 0.9


In [20]:
# Saving locally without committing
from IPython.display import FileLink

FileLink('result.mp4')


/kaggle/working/result.mp4